In [1]:
import pandas as pd
import numpy as np
import json
from random import uniform
from decimal import Decimal
from time import strftime, localtime, sleep
from datetime import datetime
from pandas import ExcelWriter
from random import random

In [2]:
Start_Skip_Range = 5000
Stop_Skip_Range  = 6000

ExcelFilePath = 'D:\\Downloads\\motor_rdpr_mambakkam_savings (1).xlsx'
ReadExcelSheetName = 'Sept 2022 -  Real time data'

Timestamp_Column = 'Created Date'

MissingDataFilledSheetName = "Sept2022_ALLFilled_Data_0"
NonContinuousDataSheetName = "Sept2022_NonCont_Data_0"

In [3]:
def Update_Dict(final_dict, Debug_Flag,mn):

    break_int = 0
    nf_int    = 0
    Curr_Dict = None
    Prev_Dict = dict()
    updated_dict = dict()
    
    for idx, metric_dict in final_dict.items():
        Curr_Dict = metric_dict

        New_Formed_Row_Dict = None        
        if len(Prev_Dict) == 0:
            New_Formed_Row_Dict = None
            updated_dict[str(Curr_Dict[Timestamp_Column+"Epoch"])] = Curr_Dict 
        else:
            Curr_Dict = dict()
            for k,v in metric_dict.items():
                try:
                    if (float(str(v))) == 0 :
                        Curr_Dict[k] = Prev_Dict[k]
                    else:
                        Curr_Dict[k] = v
                except:
                    Curr_Dict[k] = v
                    pass
            
            if Curr_Dict[Timestamp_Column+"Epoch"] - Prev_Dict[Timestamp_Column+"Epoch"] >= 120000:
                New_Formed_Row_Dict = dict()
                nf_int+=1
                for k2 in Curr_Dict.keys():
                    
                    Curr_Val = Curr_Dict[k2]
                    try:
                        Prev_Val = Prev_Dict[k2]
                    except:
                        print (Prev_Dict)
                        raise
                    
                    if type(Curr_Val) is str:
                        New_Formed_Row_Dict[k2] = Prev_Val
                    else:
                        d = Decimal(str(Curr_Val))
                        round_off_val = -1 * d.as_tuple().exponent
    
                        if k2 == Timestamp_Column+"Epoch":
                            if round_off_val == 0:
                                New_Formed_Row_Dict[k2] = int(uniform(Prev_Val+60000, Prev_Val+105000))
                            else:
                                New_Formed_Row_Dict[k2] = round(uniform(Prev_Val+60000, Prev_Val+105000), round_off_val)
                        else:
                            if round_off_val == 0:
                                New_Formed_Row_Dict[k2] = int(uniform(Prev_Val, Prev_Val + ( -1 * ((Prev_Val-Curr_Val )*0.314)   ) ))
                            else:
                                New_Formed_Row_Dict[k2] = round(uniform(Prev_Val, Prev_Val + ( -1 * ((Prev_Val-Curr_Val )*0.314)) ), round_off_val)
                            
                New_Formed_Row_Dict[Timestamp_Column] = strftime('%Y-%m-%d %H:%M:%S', localtime(New_Formed_Row_Dict[Timestamp_Column+"Epoch"]/1000.00))+".000"
                updated_dict[str(New_Formed_Row_Dict[Timestamp_Column+"Epoch"])] = New_Formed_Row_Dict 
                updated_dict[str(Curr_Dict[Timestamp_Column+"Epoch"])] = Curr_Dict 
                #print (mn,".)", New_Formed_Row_Dict[Timestamp_Column])
            else:
                updated_dict[str(Curr_Dict[Timestamp_Column+"Epoch"])] = Curr_Dict 
             
        break_int +=1   

        for k,v in metric_dict.items():
            try:
                if (float(str(v))) == 0 :
                    None
                else:
                    Prev_Dict[k] = v
            except:
                Prev_Dict[k] = v
                pass

    if Debug_Flag:
        print ("\t\t\t", mn, datetime.now(), " New Formed :: ",nf_int)
    
    if nf_int > 0:
        return updated_dict,"Continue"
    else:
        return final_dict, "Done"

In [4]:
xls = pd.ExcelFile(ExcelFilePath)
df1 = pd.read_excel(xls, ReadExcelSheetName,skiprows=1)

df1[Timestamp_Column+'Epoch'] = (pd.to_datetime(df1[Timestamp_Column]).values.astype(np.int64) // 10 ** 6) - 19800000

df1['json'] = df1.apply(lambda x: x.to_json(), axis=1)
df2 = df1[['json']]

In [5]:
json_dict = json.loads(df2.to_json().replace("\\\\u2013","-"))['json']

formed_dict = dict()
for k,v in json_dict.items():
    formed_dict[int(k)] = json.loads(v)

In [6]:
Break_Str = "Continue"
print ("Len Now : ",len(formed_dict))
i = 0
Debug_Flag = True
while Break_Str == "Continue":
    i+=1
    formed_dict,Break_Str = Update_Dict(formed_dict,Debug_Flag,i)
    #print (len(formed_dict))
    if i%25 == 0:
        print ("\t\tIteration  : ",i)
        Debug_Flag = True
    else:
        Debug_Flag = False
print ("\tTotal Iteration : ",i)    
print ("Len Now : ",len(formed_dict))

df3 = pd.DataFrame(formed_dict).T
with pd.ExcelWriter(ExcelFilePath, mode='a',if_sheet_exists='new') as writer:
    df3.to_excel(writer, sheet_name=MissingDataFilledSheetName)

Len Now :  6000
			 1 2023-11-29 18:04:44.983030  New Formed ::  5663
		Iteration  :  25
			 26 2023-11-29 18:05:01.642962  New Formed ::  3
		Iteration  :  50
			 51 2023-11-29 18:05:19.012821  New Formed ::  1
		Iteration  :  75
			 76 2023-11-29 18:05:36.533527  New Formed ::  1
	Total Iteration :  85
Len Now :  12616


In [7]:
skip_id_list = tuple()
while len(skip_id_list) <= round( np.random.randint(Start_Skip_Range, Stop_Skip_Range) ):
    skip_id_list = skip_id_list + ( round(np.random.randint(1,len(formed_dict))), )
    if len(skip_id_list) % 500 == 0:
        skip_id_list = tuple(set(skip_id_list))
len(skip_id_list)

Non_Cont_DataDict = dict()
i = 0
for k,v in formed_dict.items():
    i+=1
    if i in skip_id_list:
        None
    else:
       Non_Cont_DataDict[k] = v 

df4 = pd.DataFrame(Non_Cont_DataDict).T

with pd.ExcelWriter(ExcelFilePath, mode='a',if_sheet_exists='new') as writer:
    df4.to_excel(writer, sheet_name=NonContinuousDataSheetName)